In [ ]:
!pip install -U langchain-openai

In [17]:
import pandas as pd
import numpy as np
import time
import pathlib
import os
import re

from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from langchain_openai import OpenAIEmbeddings

from typing import List
from dotenv import load_dotenv

from langchain.vectorstores import Chroma

# Load environment variables

In [18]:
path_env = pathlib.Path('/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env')

load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key

## Create VectorDB with parquet docs

In [ ]:
# Se ha probado con outsiders (pero el objetivo será usarlos con las licitaciones completas de BSC)
path = '/export/usuarios_ml4ds/cggamella/NP-Company-Process/data/DESCARGAS_ENTREGABLES/outsiders.parquet'
df_out = pd.read_parquet(path)

def unify_colname(col):
    return ".".join([el for el in col if el])

df_out.columns = [unify_colname(col) for col in df_out.columns]

index_names = df_out.index.names
#Se resetea el índice sobre el propio dataframe
df_out.reset_index(inplace=True)
#Crear identifier con los index_names separados con '/', así generamos un id único para cada fila
df_out["identifier"] = df_out[index_names].astype(str).agg("/".join, axis=1)
#Filtrado para quedarme con esas 2 cols
df_out = df_out[['identifier', 'title']]

# Normalizar el campo textual para evitar problemas
df_out['title'] = df_out['title'].str.lower()

In [ ]:
# Crear el text splitter, [chunk_size: #caracteres de cada chunk];
#[chunk_overlap: #caracteres solapan entre chunks para no perder info.]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Crear una lista para almacenar los documentos
documents = []
# Procesar cada fila como un documento separado
for idx, row in df_out.iterrows():
    example_document = row['title']
    #print("el example doc es:",example_document)
    doc = Document(page_content=example_document, metadata={"url": "local", "source": "initial", "identifier": row['identifier']})
    #print("El doc es:",doc)
    # Dividir el documento en fragmentos
    chunks = text_splitter.split_text(doc.page_content)
    #print("Los chunks son:",chunks)
    for chunk in chunks:
        chunk_doc = Document(page_content=chunk, metadata=doc.metadata)
        #print("EL chunk_doc es:\n",chunk_doc)
        documents.append(chunk_doc)

# Crear bbdd vectorial con Chroma y guardar embeddings

In [118]:
# Embed and store the texts
path_to_index = '/export/usuarios_ml4ds/cggamella/NP-Search-Tool/aux_scripts/RAG'
# Supplying a persist_directory will store the embeddings on disk
persist_directory = (pathlib.Path(path_to_index) / 'db1').as_posix()

In [119]:
start = time.time()

# Define embeddings
embeddings = OpenAIEmbeddings()

# Almacenar los fragmentos en una base de datos vectorial usando Chroma
# Se extrae el contenido (page_content).El contenido se pasa a OpenAIEmbeddings
# para obtener los embeddings. El vector resultante se almacena en la base de datos junto con los metadatos(índices).
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory 
)

# Tiempo total de ejecución, tarda 51 mins en 300k docs
end = time.time()
print(f"Total time is {end - start} seconds")

Total time is 2886.445102930069 seconds


In [120]:
#Hay que hacerlo para guardar de manera persistente en el disco, SOLO EN NOTEBOOKS, en scripts no es necesario.
vectordb.persist()
vectordb = None

/home/cggamella/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [121]:
#Cargar la bbdd vectorial de Chroma(ya creada) que contiene docs, embeddings y metadatos 'identifier'
vectordb = Chroma(persist_directory=persist_directory)

In [122]:
results = vectordb.get(limit=6, include=['documents', 'embeddings', 'metadatas'])

In [123]:
results

{'ids': ['00007b08-f57c-435d-9038-2956ad47f807',
  '0000816e-ff3b-4b9b-9de9-c2507411cd4c',
  '0000cc4a-11a2-4d54-961e-eda7e2174e4f',
  '0000f9d6-b683-4054-b19e-f9bbda23fd4e',
  '00011233-0d1a-4d79-b01a-5778bf9676db',
  '00011399-df46-4f5f-9cab-00a3bb8b440b'],
 'embeddings': [[-0.011149450205266476,
   -0.014979518949985504,
   -0.0063993483781814575,
   -0.008593800477683544,
   -0.0053021227940917015,
   0.010692841373383999,
   -0.014829587191343307,
   0.003479091916233301,
   -0.006331197917461395,
   -0.014829587191343307,
   -0.014543354511260986,
   -0.0036665063817054033,
   -0.008723286911845207,
   -0.01960013434290886,
   -0.022312531247735023,
   -0.012362532317638397,
   -0.005779177416115999,
   -0.0014209415530785918,
   0.02330753207206726,
   0.010999519377946854,
   -0.020526982843875885,
   -0.009084485471248627,
   0.00047066560364328325,
   -0.039581913501024246,
   -0.0022200082894414663,
   -0.01774643547832966,
   0.016206230968236923,
   -0.020526982843875885,


# Retriever personalizado

In [9]:
'''
class AcronymAwareRetriever(BaseRetriever):
    """
    Retriever personalizado para buscar documentos que contienen un acrónimo específico.
    """
    documents: List[Document]
    k: int

    def _get_relevant_documents(
        self, acronym: str, *, run_manager: CallbackManagerForRetrieverRun = None
    ) -> List[Document]:
        """
        Recupera documentos que contienen el acrónimo proporcionado.
        """
        matching_documents = []
        pattern = re.compile(r'\b{}\b'.format(re.escape(acronym)), re.IGNORECASE)

        for document in self.documents:
            if len(matching_documents) >= self.k:
                break
            if pattern.search(document.page_content):
                matching_documents.append(document)
        
        return matching_documents
'''

In [28]:
class AcronymAwareRetriever(BaseRetriever):
    """
    Custom retriever for searching documents containing a specific acronym.
    """
    vectorstore: Chroma
    k: int

    def _get_relevant_documents(
        self, acronym: str, *, run_manager: CallbackManagerForRetrieverRun = None
    ) -> List[str]:
        """
        Retrieve documents containing the provided acronym by directly querying
        the vector database.
        """
        # Utilizar la bbdd vectorial para buscar documentos que contengan el acrónimo
        search_query = {"$contains": acronym}
        results = self.vectorstore.get(where_document=search_query, limit=self.k)
        #Devuelve diccionario con metadatas, y selecciono la info de los documentos
        documents = results['documents']

        return documents

In [29]:
acronym_retriever = AcronymAwareRetriever(vectorstore=vectordb, k=5)
result_docs = acronym_retriever._get_relevant_documents("ghz")

# Imprimir cada documento
for doc in result_docs:
    print(doc)
    print('*'*100)

el procesador alcanza velocidades de procesamiento de hasta 4 ghz, garantizando eficiencia.
****************************************************************************************************


In [30]:
# Tu prompt_template
prompt_template_text = """
Given the following {summaries}. These documents are related or have the context to understand the following task:
Given the word "{acronym}", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.
And after reading the previous documents I sent you, where spaces between words are replaced by underscores '_'.
If the expanded word is too long or if there is any other issue, do NOT provide any answer. 
--------------------
Please give me your answer in the format 'acronym_word:expanded_word'.
For example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", the answer must be: aeat:agencia_estatal_de_administración_tributaria 
"""
prompt_template = PromptTemplate.from_template(template=prompt_template_text)

In [31]:
# Set up the turbo LLM
llm = ChatOpenAI(
    temperature=0.2,
    model_name='gpt-4o'
)
# Crear el retriever personalizado con una lista de documentos
acronym_retriever = AcronymAwareRetriever(vectorstore=vectordb, k=3)

# Crear la chain
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    retriever=acronym_retriever,
    chain_type="stuff",
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt_template,
    }
)

In [32]:
def generate_acronym_expansions(chain, acronyms):
    results = {}
    for acronym in acronyms:
        # Recuperar documentos relevantes utilizando el retriever
        summaries = chain.retriever.get_relevant_documents(acronym)
        print("Los RETRIEVED DOCS SON:\n", summaries)
        # Formar el prompt utilizando el template
        prompt = prompt_template.format(acronym=acronym, summaries=summaries)
        # Ejecutar la cadena con el prompt usando invoke
        response = chain.invoke({"question": prompt, "acronym": acronym, "summaries": summaries})
        print(f"Respuesta del modelo para '{acronym}':\n{response}\n")
        # Obtener la respuesta del modelo de lenguaje
        answer = response['answer']
        print(f"LA ANSWER ES:\n",answer)
        # Parsear la respuesta y agregarla a los resultados
        results[acronym] = answer.strip()
    return results

In [182]:
# Lista de acrónimos a buscar
acronyms = ['ghz','ecc','vga','s.l', 's.a.', 'bop', 'pcap', 'ceip', 'jjmm', 'smp', 'avda', 'ffcc', 'itv']

# Generar las expansiones de los acrónimos
acronym_expansions = generate_acronym_expansions(chain, acronyms)

Los RETRIEVED DOCS SON:
 ["l'objecte del contracte és l'adquisició d'un perfilador vertical doppler (radar meteorològic) de banda k (24 ghz) per a l'estudi de perfils de precipitació atmosfèrica amb elevada resolució espacial i temporal (1 minut i 50 m respectivament), model mrr-pro del fabricant metek.\n\naquest equip permet als investigadors realitzar els estudis descrits al projecte ondas de gravedad, precipitacion orográfica y procesos asociados en áreas de montaña contemplat a la sol·licitud de pròrroga del projecte de recerca concedit pel ministerio de economía y competitividad amb número de referència cgl2015-65627-c3-2-r (mineco/feder).", "subministrament d'un generador vectorial de 20 ghz amb modulacio i/q de 2 ghz d'ample de banda i 2g mostres de memòria per al departament de teoria del senyal i comunicacions de la upc.", 'servei de manteniment preventiu de l’equip de rmn bruker avance neo d’1 ghz en els termes establerts al plec de prestacions tècniques.\nl’equip objecte de 


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'ecc':
{'question': '\nGiven the following ["transmissió onerosa i constitució d\'un dret de superfície sobre la finca municipal situada a l\'avigunda sitges 13 cantonada amb carrer camp dels prats núm. 11-15 per a destinar-la a la promoció, construcció i posterior gestió d\'habitatges de protecció oficial en règim de lloguer assequible.", "1.\\tl\'objecte del contracte és el servei de neteja i desinfecció interior de contenidors de càrrega posterior de les fraccions de paper-cartró, vidre, envasos, orgànica i resta amb capacitats que van des dels 120 litres fins a 1.100 litres.\\n\\nen el plec de prescripcions tècniques, en endavant ppt, es defineixen les tasques a realitzar en el servei, tipus i quantitats de contenidors objecte del contracte, freqüència del servei, horaris, característiques dels vehicles, característiques dels accessoris, mitjans materials i humans a emprar.\\n\\n2.\\tles especificaci

Los RETRIEVED DOCS SON:
 ['suministro de ambisome 50 mg., medicamentos de distribución exclusiva de la firma "gilead sciences s.l.” para el tratamiento de pacientes con neutropenia grave con destino al servicio de farmacia, del hospital universitario 12 de octubre.', 'suministro medicamentos exclusivos teva pharma s.l.u. a adjudicar por procedimiento negociado sin publicidad. pnsp-sum 035/21', 'suministro de medicamentos de distribución exclusiva de la firma orphan europe s.l., con destino al servicio de farmacia del hospital universitario 12 de octubre']


> Entering new RetrievalQAWithSourcesChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following . These documents are related or have the context to understand the following task:
Given the word "s.l", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining th



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following . These documents are related or have the context to understand the following task:
Given the word "bop", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.
And after reading the previous documents I sent you, where spaces between words are replaced by underscores '_'.
If the expanded word is too long or if there is any other issue, do NOT provide any answer. 
--------------------
Please give me your answer in the format 'acronym_word:expanded_word'.
For example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", the answer must be: aeat:agencia_estatal_de_administración_tributaria 


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo pa


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'pcap':
{'question': '\nGiven the following ["l\'objecte del contracte és la prestació del servei de transport adaptat per a persones amb discapacitat des del baix empordà a centres de serveis socials d\'altres comarques, en concret, dos rutes (anada i tornada) que s\'indiquen a l\'annex 1 del pcap.", \'constitueix l’objecte d’aquest contracte el subministrament i instal·lació d’aire condicionat als espais comuns de la residència i centre de dia montsacopa per garantir una temperatura òptima de confort en períodes de calor. (veure clàusula 1 del pcap)\', "realització d\'una enquesta telefònica a una mostra de 1.500 i amb una durada màxima de 20 minuts en les condicions que es determinen en el document de concreció de condicions (cc) i en el pcap i pptp de l\'acord marc de serveis per a la realització d\'estudis d\'opinió d\'interès de la generalitat (ceo-2021-17). lot 2 de l\'acord marc."]. These documen

Los RETRIEVED DOCS SON:
 ["equip per la mesura i monitorització de senyals de vídeo i audio sobre ip amb l'estàndard smpte 2110, senyal de temps ptp i vídeo sdi-4k", "serveis per l'assessorament, el desenvolupament i l'execució del social media plan (en endavant smp) i més concretament de la gestió de les xarxes socials de catalunya experience pel mercat que correspongui segons el lot:\n-\tlot 1: frança\n-\tlot 2: gran bretanya i irlanda\n-\tlot 3: europa central\n-\tlot 4: països baixos i bèlgica\n-\tlot 5: itàlia \n-\tlot 6: rússia i països de l'est\n-\tlot 7: estats units i canadà\n-\tlot 8: brasil i argentina\n-\tlot 9: àsia - pacific, excepte xina\n-\tlot 10: xina\n-        lot 11: pirineus", "serveis per l'assessorament, el desenvolupament i l'execució del social media plan (en endavant smp) i més concretament de la gestió de les xarxes socials de catalunya experience pel mercat que correspongui segons el lot.\n\nl'objecte del contracte es divideix en els lots següents: \n- lot 1

Los RETRIEVED DOCS SON:
 ['contratación del mantenimiento preventivo, correctivo, evolutivo y proactivo de sistemas de peaje de euskotren en el ferrocarril (ffcc), funicular y en tranvías', 'mantenimiento de los equipos del sistema de peaje de euskotren en el ffcc y el tranvía de bilbao', 'urbanización parcial de la acera de los números impares de la avda. cataluña entre ffcc a huesca y la c/ rio ara']


> Entering new RetrievalQAWithSourcesChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following . These documents are related or have the context to understand the following task:
Given the word "ffcc", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.
And after reading the previous documents I sent you, where spaces between words are replaced by 

In [183]:
acronym_expansions

{'ghz': 'ghz:gigahertz',
 'ecc': 'ecc:electronic_control_center',
 'vga': 'vga:video_graphics_array',
 's.l': 's.l:sociedad_limitada',
 's.a.': 'Sure, please provide the acronyms you need expanded.',
 'bop': 'bop:business_operations_plan',
 'pcap': 'pcap:packet_capture',
 'ceip': 'ceip:centro_de_educación_infantil_y_primaria',
 'jjmm': 'I\'m sorry, but based on the information provided, I cannot determine an appropriate expanded word for the acronym "jjmm".',
 'smp': 'smp:symmetric_multiprocessing',
 'avda': 'avda:avenida',
 'ffcc': 'ffcc:ferrocarriles',
 'itv': 'itv:inspección_técnica_de_vehículos'}

# NP-Search-Tool sacar acrónimos con RAG

In [ ]:
#!pip install chromadb
#!pip install -U langchain-openai

In [54]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from sentence_transformers import SentenceTransformer
import re

In [55]:
path = '/export/usuarios_ml4ds/cggamella/NP-Company-Process/data/DESCARGAS_ENTREGABLES/outsiders.parquet'
df_out = pd.read_parquet(path)

In [56]:
def unify_colname(col):
    return ".".join([el for el in col if el])

In [57]:
df_out.columns = [unify_colname(col) for col in df_out.columns]

In [58]:
#Para coger ['zip', 'file name', 'entry']
index_names = df_out.index.names
#Se resetea el índice sobre el propio dataframe
df_out.reset_index(inplace=True)
#Pone como identifier los index names separados con '/', así generamos un id único para cada fila
df_out["identifier"] = df_out[index_names].astype(str).agg("/".join, axis=1)

In [59]:
df_filtered = df_out.loc[:100, ['identifier', 'title']]

In [60]:
# Calcular la longitud media de la columna 'title'
df_filtered['title_length'] = df_filtered['title'].apply(len)
average_length = df_filtered['title_length'].mean()
average_length

151.6039603960396

In [61]:
# Inicializar el modelo de embeddings
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device='cuda')
# Generar embeddings para cada título en el dataframe
embeddings = model.encode(df_out['title'].tolist())

In [62]:
def retrieve_context_similarity(acronym, embeddings, df_out):
    acronym_embedding = model.encode([acronym])
    #print("La query embedding es:\n", query_embedding)
    similarities = cosine_similarity(acronym_embedding, embeddings)[0]
    print("Las cosine similarities son:\n", similarities)
    most_similar_indices = np.argsort(similarities)[-10:][::-1]
    print("Los indices mas similares son:\n", most_similar_indices)
    results = [df_out['title'].iloc[i] for i in most_similar_indices]
    print("Los resultados del retrieve son:\n", results)
    return results

In [63]:
def retrieve_context_contain_acronym(acronym, df_out):
    # Crear el patrón de expresión regular para coincidir con la palabra exacta
    pattern = re.compile(r'\b' + re.escape(acronym) + r'\b', re.IGNORECASE)
    # Filtrar los títulos que contienen el acrónimo exacto
    results = df_out[df_out['title'].str.contains(pattern, na=False)]
    # Obtener los títulos y limitar a los primeros 5 documentos
    titles = results['title'].head(5).tolist()
    # Limitar cada título a los primeros 70 caracteres
    limited_titles = [title[:70] for title in titles]
    return limited_titles

In [69]:
def retrieve_context_contain_acronym_improved(acronym, df_out):
    # Crear el patrón de expresión regular para coincidir con la palabra exacta
    pattern = re.compile(r'\b' + re.escape(acronym) + r'\b', re.IGNORECASE)
    contexts = []
    
    # Iterar sobre cada título en el DataFrame
    for title in df_out['title']:
        if pd.isna(title):
            continue
        # Buscar todas las coincidencias del patrón en el título
        for match in re.finditer(pattern, title):
            # Asegurarse de no ir más allá del comienzo del texto
            start = max(match.start() - 70, 0)  
            end = match.end() + 70  
            # Extraer el contexto alrededor del acrónimo
            context = title[start:end]
            contexts.append(context)
            # Limitar a los primeros 5 documentos
            if len(contexts) >= 5:  
                return contexts
    
    return contexts

In [71]:
llm = OpenAI()
             
def generate_equivalent_expression(acronym):
    contexts = retrieve_context_contain_acronym_improved(acronym, df_out)    
    context_texts = " ".join(contexts)
    prompt = f"Given the acronym {acronym}, and the following context: {context_texts}, provide the large expression of the acronym. Please provide me the result in a dictionary format 'acronym:equivalence'"
    print("EL PROMPTING ES:", prompt)
    response = llm.generate([prompt])
    print("La respuesta es:", response)
    return response.generations[0][0].text 

In [72]:
acronyms = ['ghz','ecc','vga','s.l', 's.a.', 'bop', 'pcap', 'ceip', 'jjmm', 'smp', 'avda', 'ffcc', 'itv']  

In [73]:
results = {}
for acronym in acronyms:
    expression = generate_equivalent_expression(acronym)
    results[acronym] = expression

EL PROMPTING ES: Given the acronym ghz, and the following context: s:

Chipset: Intel® Q270 
Processador Intel® Core¿ i5 7ª Gen 7500 3.4 Ghz (3.8 Ghz) (Quad Core)
Placa gràfica HD 530
Sistema operatiu Microsoft et: Intel® Q270 
Processador Intel® Core¿ i5 7ª Gen 7500 3.4 Ghz (3.8 Ghz) (Quad Core)
Placa gràfica HD 530
Sistema operatiu Microsoft Windows   d'un perfilador vertical Doppler (radar meteorològic) de banda K (24 GHz) per a l'estudi de perfils de precipitació atmosfèrica amb elevada re connexió mitjançant una xarxa de radiofreqüència en banda lliure de 5 GHz en tecnologia basada en 802.11a, 802.11n, 802.11ac o bé 802.16 per la Subministrament d'un generador vectorial de 20 Ghz amb modulacio I/Q de 2 GhZ d'ample de banda i 2G mostres de memòria p, provide the large expression of the acronym. Please provide me the result in a dictionary format 'acronym:equivalence'
La respuesta es: generations=[[Generation(text='\n\nGHz: gigahertz', generation_info={'finish_reason': 'stop', 'logp

La respuesta es: generations=[[Generation(text='\n\nCEIP: Centro de Educación Infantil y Primaria (Center for Infant and Primary Education)', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 241, 'completion_tokens': 21, 'prompt_tokens': 220}, 'model_name': 'gpt-3.5-turbo-instruct'} run=[RunInfo(run_id=UUID('fffc43b3-4187-4526-baf6-cf692be4e09b'))]
EL PROMPTING ES: Given the acronym jjmm, and the following context: a de 50 metres per construir en l'àmbit de l'anella mediterrània dels JJMM 2018, aprovat definitivament pel Consell Plenari de data 28 de novemb, provide the large expression of the acronym. Please provide me the result in a dictionary format 'acronym:equivalence'
La respuesta es: generations=[[Generation(text='\n\nJJMM: Jocs del Mediterrani del 2018', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 97, 'completion_tokens': 13, 'prompt_tokens': 84}, 'model_nam

In [74]:
results

{'ghz': '\n\nGHz: gigahertz',
 'ecc': "\n\n'ECC: Error Checking and Correction'",
 'vga': '\n\nvga: Video Graphics Array',
 's.l': '\n\nS.L: Sociedad Limitada (Spanish for Limited Company)',
 's.a.': '\n\ns.a.: Sociedad Anónima (Spanish for "public limited company")',
 'bop': "\n'bop:Butlletí Oficial de la Província'",
 'pcap': '\n\npcap: Project Capture Plan',
 'ceip': '\n\nCEIP: Centro de Educación Infantil y Primaria (Center for Infant and Primary Education)',
 'jjmm': '\n\nJJMM: Jocs del Mediterrani del 2018',
 'smp': '\n\nSMP: Social Media Plan',
 'avda': '\n\nAVDA: Avenida',
 'ffcc': '\n\nffcc: Ferrocarrils de la Generalitat de Catalunya (FFCC)',
 'itv': '\n\nITV: Inspección Técnica Periódica del Vehículo'}

# PRUEBAS PARA EVALUAR MÉTODO

In [12]:
import pandas as pd
import ast  # Módulo para evaluar cadenas como estructuras de datos de Python seguras

# Ruta del archivo
path = '/export/usuarios_ml4ds/cggamella/RAG_tool/dataset_pruebas.xlsx'
df = pd.read_excel(path)

In [35]:
lista_acronyms = set(df['acronyms'].str.lower().tolist())

In [36]:
lista_acronyms

{'aena',
 'ai',
 'avda',
 'bme',
 'boe',
 'ccaa',
 'ceip',
 'ceo',
 'cfo',
 'cia, fbi',
 'cia, nsa ',
 'cnmv',
 'css',
 'cto',
 'faq',
 'fda',
 'ffcc',
 'fmi',
 'ghz',
 'gps',
 'html',
 'http',
 'iot',
 'itv',
 'jpeg',
 'lan',
 'minhac',
 'ml',
 'nasa',
 'nasa, esa',
 'nato, otan',
 'ocde',
 'oms,covid',
 'once',
 'ong',
 'ong, acnur',
 'ong, greenpeace',
 'onu',
 'pcap',
 'pdf',
 'php',
 'pin',
 'pk',
 'pyme',
 'ram',
 'renfe',
 'rne',
 'rrhh',
 's.a.',
 'seat',
 'seo',
 'sepe',
 'sms',
 'sql',
 'tv3',
 'ue',
 'ue, otan',
 'unesco',
 'unicef',
 'upc',
 'url',
 'usb',
 'vp',
 'vpn',
 'wifi'}

In [19]:
# Normalizar el campo textual para evitar problemas
df['text'] = df['text'].str.lower()

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Crear una lista para almacenar los documentos
documents = []
# Procesar cada fila como un documento separado
for idx, row in df.iterrows():
    example_document = row['text']
    print("el example doc es:",example_document)
    doc = Document(page_content=example_document, metadata={"url": "local", "source": "initial"})
    print("El doc es:",doc)
    # Dividir el documento en fragmentos
    chunks = text_splitter.split_text(doc.page_content)
    print("Los chunks son:",chunks)
    for chunk in chunks:
        chunk_doc = Document(page_content=chunk, metadata=doc.metadata)
        print("EL chunk_doc es:\n",chunk_doc)
        documents.append(chunk_doc)

el example doc es: las ong son organizaciones no gubernamentales que trabajan en proyectos sociales y medioambientales.
El doc es: page_content='las ong son organizaciones no gubernamentales que trabajan en proyectos sociales y medioambientales.' metadata={'url': 'local', 'source': 'initial'}
Los chunks son: ['las ong son organizaciones no gubernamentales que trabajan en proyectos sociales y medioambientales.']
EL chunk_doc es:
 page_content='las ong son organizaciones no gubernamentales que trabajan en proyectos sociales y medioambientales.' metadata={'url': 'local', 'source': 'initial'}
el example doc es: la cia y el fbi colaboran en investigaciones internacionales de seguridad y espionaje.
El doc es: page_content='la cia y el fbi colaboran en investigaciones internacionales de seguridad y espionaje.' metadata={'url': 'local', 'source': 'initial'}
Los chunks son: ['la cia y el fbi colaboran en investigaciones internacionales de seguridad y espionaje.']
EL chunk_doc es:
 page_content=

In [23]:
# Embed and store the texts
path_to_index = '/export/usuarios_ml4ds/cggamella/NP-Search-Tool/aux_scripts/RAG'
# Supplying a persist_directory will store the embeddings on disk
persist_directory = (pathlib.Path(path_to_index) / 'db_pruebas').as_posix()

In [24]:
start = time.time()

# Define embeddings
embeddings = OpenAIEmbeddings()

# Almacenar los fragmentos en una base de datos vectorial usando Chroma
# Se extrae el contenido (page_content).El contenido se pasa a OpenAIEmbeddings
# para obtener los embeddings. El vector resultante se almacena en la base de datos junto con los metadatos(índices).
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory 
)

# Tiempo total de ejecución, tarda 51 mins en 300k docs
end = time.time()
print(f"Total time is {end - start} seconds")

Total time is 3.4274675846099854 seconds


In [25]:
#Cargar la bbdd vectorial de Chroma(ya creada) que contiene docs, embeddings y metadatos 'identifier'
vectordb = Chroma(persist_directory=persist_directory)

In [26]:
results = vectordb.get(limit=1, include=['documents', 'embeddings'])

In [41]:
acronym_retriever = AcronymAwareRetriever(vectorstore=vectordb, k=3)
# Recorrer cada acrónimo en la lista
for acronym in lista_acronyms:
    result_docs = acronym_retriever._get_relevant_documents(acronym)
    
    # Imprimir cada documento y separarlos con una línea de asteriscos
    for doc in result_docs:
        print("Para el acronimo:",acronym)
        print(doc)
        print('*' * 100)

Para el acronimo: s.a.
la empresa opera bajo la forma jurídica de s.a., asegurando transparencia y responsabilidad.
****************************************************************************************************
Para el acronimo: itv
recuerde realizar la itv de su vehículo para asegurar su buen estado mecánico y seguridad.
****************************************************************************************************
Para el acronimo: vp
la vpn protege la privacidad en línea al encriptar la conexión y ocultar la dirección ip del usuario, asegurando la navegación segura en redes públicas.
****************************************************************************************************
Para el acronimo: aena
aena, responsable de la gestión aeroportuaria en españa, anunció mejoras en la seguridad y servicios para pasajeros en todos sus aeropuertos.
****************************************************************************************************
Para el acronimo: onu
la onu

In [37]:
acronym_expansions = generate_acronym_expansions(chain, lista_acronyms)

Los RETRIEVED DOCS SON:
 ['la empresa opera bajo la forma jurídica de s.a., asegurando transparencia y responsabilidad.']


> Entering new RetrievalQAWithSourcesChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following . These documents are related or have the context to understand the following task:
Given the word "s.a.", understand that an acronym word is a type of abbreviation formed by taking the initial
letters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.
And after reading the previous documents I sent you, where spaces between words are replaced by underscores '_'.
If the expanded word is too long or if there is any other issue, do NOT provide any answer. 
--------------------
Please give me your answer in the format 'acronym_word:expanded_word'.
For example, for the following acronyms, "aeat" stands for "agencia_estatal_de_adm


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'vp':
{'question': '\nGiven the following [\'la vpn protege la privacidad en línea al encriptar la conexión y ocultar la dirección ip del usuario, asegurando la navegación segura en redes públicas.\']. These documents are related or have the context to understand the following task:\nGiven the word "vp", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the format \'acronym_word:expanded_word\'.\nFor example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'onu':
{'question': '\nGiven the following [\'la onu busca mantener la paz y seguridad internacionales.\']. These documents are related or have the context to understand the following task:\nGiven the word "onu", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the format \'acronym_word:expanded_word\'.\nFor example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", the answer must be: aeat:agencia_estatal_de_administración_tributaria \n', 'acr


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'usb':
{'question': '\nGiven the following [\'conecta tu memoria usb al puerto del ordenador para transferir rápidamente archivos importantes o realizar copias de seguridad de tus documentos.\']. These documents are related or have the context to understand the following task:\nGiven the word "usb", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the format \'acronym_word:expanded_word\'.\nFor example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_trib


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'jpeg':
{'question': '\nGiven the following [\'la galería de fotos está en formato jpeg para garantizar la mejor calidad de imagen sin ocupar demasiado espacio en el almacenamiento.\']. These documents are related or have the context to understand the following task:\nGiven the word "jpeg", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the format \'acronym_word:expanded_word\'.\nFor example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", 

...


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'upc':
{'question': '\nGiven the following [\'la upc lanzó un nuevo programa de becas para estudiantes de ingeniería, promoviendo la innovación y la excelencia académica.\', \'la conexión wifi del hotel es rápida y estable, permitiendo a los huéspedes trabajar y disfrutar de streaming sin interrupciones durante su estancia.\']. These documents are related or have the context to understand the following task:\nGiven the word "upc", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the form


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'ai':
{'question': '\nGiven the following []. These documents are related or have the context to understand the following task:\nGiven the word "ai", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the format \'acronym_word:expanded_word\'.\nFor example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", the answer must be: aeat:agencia_estatal_de_administración_tributaria \n', 'acronym': 'ai', 'summaries': [], 'answer': 'ai:inteligencia_artifi


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'pcap':
{'question': '\nGiven the following [\'en el pcap  establecido para el proyecto garantiza el cumplimiento estricto de normativas ambientales.\']. These documents are related or have the context to understand the following task:\nGiven the word "pcap", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the format \'acronym_word:expanded_word\'.\nFor example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", the answer must be: aeat:agencia


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'cnmv':
{'question': '\nGiven the following []. These documents are related or have the context to understand the following task:\nGiven the word "cnmv", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the format \'acronym_word:expanded_word\'.\nFor example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", the answer must be: aeat:agencia_estatal_de_administración_tributaria \n', 'acronym': 'cnmv', 'summaries': [], 'answer': 'cnmv:comisión_na


> Finished chain.

> Finished chain.

> Finished chain.
Respuesta del modelo para 'cfo':
{'question': '\nGiven the following []. These documents are related or have the context to understand the following task:\nGiven the word "cfo", understand that an acronym word is a type of abbreviation formed by taking the initial\nletters or parts of words from a phrase or term and combining them to form a new word, that is what we call expanded_word.\nAnd after reading the previous documents I sent you, where spaces between words are replaced by underscores \'_\'.\nIf the expanded word is too long or if there is any other issue, do NOT provide any answer. \n--------------------\nPlease give me your answer in the format \'acronym_word:expanded_word\'.\nFor example, for the following acronyms, "aeat" stands for "agencia_estatal_de_administración_tributaria", the answer must be: aeat:agencia_estatal_de_administración_tributaria \n', 'acronym': 'cfo', 'summaries': [], 'answer': 'cfo:chief_financial

In [38]:
acronym_expansions

{'s.a.': 'Sure, please provide the acronyms you need expanded.',
 'ue, otan': 'ue:otan',
 'itv': 'itv:inspección_técnica_de_vehículos',
 'tv3': 'tv3:televisió_de_catalunya_3',
 'vp': 'vp:vice_president',
 'aena': 'aena:aeropuertos_españoles_y_navegación_aérea',
 'nato, otan': 'nato:organización_del_tratado_del_atlántico_norte',
 'oms,covid': 'oms:organización_mundial_de_la_salud\ncovid:coronavirus_disease',
 'onu': 'onu:organización_de_las_naciones_unidas',
 'sql': 'sql:structured_query_language',
 'ccaa': 'ccaa:comunidades_autónomas',
 'seat': 'seat:seat',
 'usb': 'usb:universal_serial_bus',
 'pyme': 'pyme:pequeña_y_mediana_empresa',
 'ong': 'ong:organización_de_las_naciones_unidas',
 'pin': 'pin:personal_identification_number',
 'jpeg': 'jpeg:joint_photographic_experts_group',
 'bme': 'bme:biomedical_engineering',
 'iot': 'iot:internet_of_things',
 'ceip': 'ceip:centro_de_educación_infantil_y_primaria',
 'ffcc': 'ffcc:ferrocarriles',
 'ong, greenpeace': 'ong:greenpeace',
 'http': 'ht